<a href="https://colab.research.google.com/github/incubusV/colab/blob/main/ARCGIS_REST_GET_TO_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv

# Set the URL for the ArcGIS REST API endpoint
url = "https://gisadaptor.mea.or.th/mapmea/rest/services/CR_SL_BKK/Streetlight_Bangkok_26082022/FeatureServer/0/query"

# Open a CSV file for writing
with open("features.csv", "w", newline="") as f:
    # Set the initial result offset
    result_offset = 0

    # Loop through all the records
    while True:
        # Set the parameters for the API call
        params = {
            "where": "1=1",
            "outFields": "*",
            "returnGeometry": "true",
            "resultRecordCount": "1000",
            "resultOffset": result_offset,
            "f": "json"
        }

        # Make the API call using the requests library
        response = requests.get(url, params=params)

        # Convert the response to a JSON object
        data = response.json()

        # Get the field names from the response
        fieldnames = [field['name'] for field in data['fields']]

        # Add a new field for the feature
        fieldnames.append("feature_x")
        fieldnames.append("feature_y")

        # Create a writer object
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        # Write the header row only on the first iteration
        if result_offset == 0:
            writer.writeheader()

        # Get the features from the response
        features = data["features"]

        # Write the features to the CSV file
        for feature in features:
            row = feature["attributes"]
            row["feature_x"] = feature["geometry"]["x"]
            row["feature_y"] = feature["geometry"]["y"]
            writer.writerow(row)

        # Increment the result offset
        result_offset += 1000

        # Check if there are more records
        if data["exceededTransferLimit"] == False:
            break
